### Import relevant libraries

In [1]:
import pandas as pd

### Load the dataset

In [2]:
%cd /content/drive/MyDrive/python_for_business_and_finance/RFM_segmentation

df = pd.read_csv("customer_data.csv")
df.head()

/content/drive/MyDrive/python_for_business_and_finance/RFM_segmentation


,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,9/8/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/9/2006,12,23


### Data cleaning and preparation

In [4]:
# Check for missing values
df.isna().sum()

customer_id          0
revenue              0
most_recent_visit    0
number_of_orders     0
recency_days         0
dtype: int64

In [5]:
# Prepare monetary variable
df["monetary"] = df["revenue"]/df["number_of_orders"]
df.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days,monetary
0,22086,777,5/14/2006,9,232,86.333333
1,2290,1555,9/8/2006,16,115,97.187500
2,26377,336,11/19/2006,5,43,67.200000
3,24650,1189,10/29/2006,12,64,99.083333
4,12883,1229,12/9/2006,12,23,102.416667


In [6]:
# Change column names to match Recency, Frequency, Monetary framework
df.rename(columns = {"number_of_orders":"frequency",
                     "recency_days":"recency"}, inplace = True)
df.head(1)

,customer_id,revenue,most_recent_visit,frequency,recency,monetary
0,22086,777,5/14/2006,9,232,86.333333


In [7]:
# Remove unnecessary columns
df.drop(columns = ["revenue", "most_recent_visit"], inplace = True)

In [10]:
# Bucket (3 buckets) customers with respect to frequency
df['F'] = pd.qcut(x = df['frequency'], q = 3, labels = [1, 2, 3] ).astype(int)

# Same for monetary
df['M'] = pd.qcut(x = df['monetary'], q = 3, labels = [1, 2, 3] ).astype(int)

# and recency
df['R'] = pd.qcut(x = df['recency'], q = 3, labels = [3, 2, 1]).astype(int) # note reversed labels

### Create RFM score

In [11]:
df['RFM'] = df['R'] + df['F'] + df['M']
df.head(1)

,customer_id,frequency,recency,monetary,F,M,R,RFM
0,22086,9,232,86.333333,2,1,1,4


### Create segmentation function

In [12]:
def segment_fn(x):
  if x >= 8:
    return "Top client"
  elif x >= 5 and x < 8:
    return "Average customer"
  else:
    return "Occasional customer"

In [13]:
df['segment'] = df['RFM'].apply(segment_fn)

df.head()

,customer_id,frequency,recency,monetary,F,M,R,RFM,segment
0,22086,9,232,86.333333,2,1,1,4,Occasional customer
1,2290,16,115,97.187500,3,2,2,7,Average customer
2,26377,5,43,67.200000,1,1,3,5,Average customer
3,24650,12,64,99.083333,3,2,3,8,Top client
4,12883,12,23,102.416667,3,2,3,8,Top client


### Aggregating by segment

In [15]:
df.groupby(by = 'segment').agg({'recency':'mean',
                                'frequency':'mean',
                                'monetary':'mean'}).round(1)

,recency,frequency,monetary
segment,,,
Average customer,171.8,9.8,97.0
Occasional customer,306.6,7.1,78.5
Top client,80.1,12.8,108.3
